In [30]:
# Here define data directory and path to metadata csv file
test = "disc1/OAS1_0027_MR1/RAW/OAS1_0027_MR1_mpr-1_anon.img"
print test.replace(".img",".hdr")
print type(test)

disc1/OAS1_0027_MR1/RAW/OAS1_0027_MR1_mpr-1_anon.hdr
<type 'str'>


In [1]:
import pandas as pd
import numpy as np
import glob, os
from operator import itemgetter

data_path='/nfs/zorba/DOWNLOADED/Oasis/'
# Below will capture all the structural and functional dicom folders (some are duplicates)
cross_glob_data = 'disc*/OAS1_*_MR1/RAW/OAS1_*_MR1_mpr-1_anon.img' # Grabs all baseline, later can add the repeat session
long_glob_data = 'OAS2_RAW_PART*/OAS2_*_MR1/RAW/mpr-1.nifti.img' # Come back and incorporate

os.chdir(data_path)

cross_scans=sorted(glob.glob(cross_glob_data)) # Focus on T1 scans for now
long_scans=sorted(glob.glob(long_glob_data))

print "length of cross scans is: " + str(len(cross_scans))
print "length of long scans is: " + str(len(long_scans))

# Now get unique subject IDs in same order as glob files
cross_data_sub_ids=[]
long_data_sub_ids=[]
for s in cross_scans:
    cross_data_sub_ids.append(s.split('/')[1])
    
for s in long_scans:
    long_data_sub_ids.append(s.split('/')[1])
    
### Read and Parse XSL
# metadata_file='/Users/haiyuancao/GitHub/xnat_import/scan-new/8100_IRI_20161115_2.csv'
metadata_cross = '/nfs/zorba/Dropbox/DOWNLOADED/Oasis/oasis_cross-sectional.csv'
metadata_long = '/nfs/zorba/Dropbox/DOWNLOADED/Oasis/oasis_longitudinal.csv'
df_cross = pd.read_csv(metadata_cross)
df_long = pd.read_csv(metadata_long)
#df['Anoymized ID'] = df['Anoymized ID'].apply(lambda x: unicode(x))
#id_list = df['Anonymized ID'].values.tolist()
df_cross = df_cross.set_index('ID')
cross_meta_sub_ids = df_cross.index.tolist()

df_long = df_long.set_index('MRI ID')
long_meta_sub_ids = df_long.index.tolist()

# Now get only unique ids for structural scans
# Go over list of paths to structural data
# Now create subject_dict that includes age, sex, and path to scan
subject_dict = {}
count=0

for i in range(len(cross_data_sub_ids)):
    item=cross_data_sub_ids[i] # Item is the subject ID "i.e. A00053278"
    T1_path=cross_scans[i]
    
    #print (item)
    #print file_path
    if item in cross_meta_sub_ids:
        #print id_list.index(item)
        count += 1
        df_index =cross_meta_sub_ids.index(item)
        age = df_cross.iloc[df_index]['Age']
        sex = df_cross.iloc[df_index]['M/F']
        
        #print (age, sex)
        subject_dict[item] = {'age': "", "sex": "", "T1_path":""}
        subject_dict[item]['age'] = age
        subject_dict[item]['sex'] = sex
        subject_dict[item]['T1_path'] = T1_path
    
print (count, len(cross_data_sub_ids))
print len(subject_dict)

for key, value in subject_dict.items()[0:10]:
    print key  
    print value["T1_path"]
    print value["age"]   
    print value["sex"]
    
############################################################################
# Here do the same for the longitudinal data
#count=0
#for i in range(len(long_data_sub_ids)):
#    item=long_data_sub_ids[i] # Item is the subject ID "i.e. A00053278"
#    T1_path=long_scans[i]
    
    #print (item)
    #print file_path
#    if item in long_meta_sub_ids:
#        count += 1
        
#        df_index = long_meta_sub_ids.index(item)
#        age = df_long.iloc[df_index]['Age']
#        sex = df_long.iloc[df_index]['M/F']
        
        #print (age, sex)
#        subject_dict[item] = {'age': "", "sex": "", "T1_path":""}
#        subject_dict[item]['age'] = age
#        subject_dict[item]['sex'] = sex
#        subject_dict[item]['T1_path'] = T1_path
    
#print (count, len(long_data_sub_ids))
#print len(subject_dict)

length of cross scans is: 416
length of long scans is: 150
(416, 416)
416
OAS1_0027_MR1
disc1/OAS1_0027_MR1/RAW/OAS1_0027_MR1_mpr-1_anon.img
43
F
OAS1_0332_MR1
disc9/OAS1_0332_MR1/RAW/OAS1_0332_MR1_mpr-1_anon.img
72
M
OAS1_0162_MR1
disc5/OAS1_0162_MR1/RAW/OAS1_0162_MR1_mpr-1_anon.img
20
F
OAS1_0270_MR1
disc7/OAS1_0270_MR1/RAW/OAS1_0270_MR1_mpr-1_anon.img
93
F
OAS1_0022_MR1
disc1/OAS1_0022_MR1/RAW/OAS1_0022_MR1_mpr-1_anon.img
69
F
OAS1_0069_MR1
disc2/OAS1_0069_MR1/RAW/OAS1_0069_MR1_mpr-1_anon.img
33
M
OAS1_0263_MR1
disc7/OAS1_0263_MR1/RAW/OAS1_0263_MR1_mpr-1_anon.img
79
M
OAS1_0267_MR1
disc7/OAS1_0267_MR1/RAW/OAS1_0267_MR1_mpr-1_anon.img
80
M
OAS1_0423_MR1
disc12/OAS1_0423_MR1/RAW/OAS1_0423_MR1_mpr-1_anon.img
75
M
OAS1_0146_MR1
disc4/OAS1_0146_MR1/RAW/OAS1_0146_MR1_mpr-1_anon.img
82
F


In [2]:
import pyxnat, os
delete_previous_scans = 1
insert_scans = 1

# Set project name
project_name="OASIS"

# connect to XNAT instance
xnat=pyxnat.Interface(config='/home/spantazatos/xnat_sp.cfg')

# Create the project in XNAT
# Create project object
project=xnat.select.project(project_name)

# First check if it already exists
if not project.exists():
    project.insert()
    
for key, value in subject_dict.items():
    print key
    print value["T1_path"]
    
    # Insert subject if not already there
    print ("Now insert subject with key: " + key)
    # create label without the MR1
    sub_label = key[:-4]
    print sub_label
    
    subject = project.subject(sub_label)
    if not subject.exists():
        subject.insert()
    
    # Since this is baseline just do MR1 for now
    session_label = "%s_%s" % (project_name,key)
    print session_label
    
    experiment=subject.experiment(session_label) #.create(experiments='xnat:mrSessionData')
   
    # Create experiment object, insert if it doesn't already exst
    if not experiment.exists():
        experiment.create() # By default it will make an xnat:mrSessionData experiment which is what we need
    
    # Now insert subject-level attributes (i.e. SEX) and experiment-level attributes (i.e. AGE)
    user_age = int(value['age'])
    if value['sex'] == 'M':
        user_sex = 'Male'
    else:
        user_sex = 'Female'
    
#     sex = 'Male'
#     weight = str(100)
#     height = str(100)
    
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/gender', user_sex)
    subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/age', str(user_age))
    #subject.attrs.set('xnat:subjectData/demographics[@xsi:type=xnat:demographicData]/height', height)
    
    model ='SIEMENS VISION'
    fs ='1.5T'
    
    experiment.attrs.set('xnat:imageSessionData/scanner',model)
    experiment.attrs.set('xnat:imageSessionData/fieldStrength',fs)

    # Below for inserting scans
    if insert_scans==1:
        scan_files=[]
        scan_files.append(value['T1_path'])
    
        # Delete old scans if desired
        if delete_previous_scans==1:
            allScans=experiment.scans()
            for scan in allScans:
                print "deleting scans"
                scan.delete()

        for ii in range(len(scan_files)):
            # ii should only be ii
            scan_type='T1'
            scan_name='T1_01'
            slice_size='8 1200'
           
            #cmd_reorient='fslreorient2std ' + value['T1_path'] + ' ' + img_reorient
            hdr_file = scan_files[ii].replace("img","hdr")
            print "new hdr file is: " + hdr_file
            
            cmd_cp1='cp ' + scan_files[ii] + ' temp.img' # Don't reorient since already including dcm2nii orientation
            os.system(cmd_cp1)
            cmd_cp2='cp ' + hdr_file + ' temp.hdr'
            os.system(cmd_cp2)
            cmd_chfiletype = 'fslchfiletype NIFTI temp'
            os.system(cmd_chfiletype)
            #cmd_reorient='fslreorient2std temp.nii anonymized_reorient.nii'
            cmd_reorient1 = 'flirt -in temp.nii -ref $T1_hires -dof 6 -omat out.mat'
            print "now flirting"
            os.system(cmd_reorient1)
            cmd_reorient2 = 'flirt -in temp.nii -out anonymized_reorient -ref temp.nii -applyxfm -init out.mat -noresample'
            os.system(cmd_reorient2)
            cmd_reorient3=c && fslorient -setqform -1 0 0 90 0 1 0 -125 0 0 1.25 -67.75 0 0 0 1 anonymized_reorient'
            os.system(cmd_reorient3)
            cmd_rm_temp='rm temp.nii'
            os.system(cmd_rm_temp)
            #cmd_gzip='gzip anonymized_reorient.nii'
            #os.system(cmd_gzip)
            
            img_reorient='anonymized_reorient.nii.gz'
            # Now insert the reoriented image
            # experiment.scan('1').resource('NIFTI').file(os.path.basename(scans_filtered[sub])).insert(
            experiment.scan(scan_name).resource('NIFTI').file(os.path.basename(img_reorient)).insert(
            img_reorient,
            content=scan_type,
            overwrite=True,
            type=scan_type,  # May need to change this to SPGR, MPRAGE, FLAIR etc?
            format='NIFTI')

            # Create ORIGINAL and SNAPSHOT to view the images in the web browser
            img_png='tmp.png'
            img_gif='tmp.gif'
            img_thumb='tmp_thumb.gif'
            cmd_create_slices='slicer ' + img_reorient + ' -S ' + slice_size + ' ' + img_png
            cmd_convert_gif='convert ' + img_png + ' ' + img_gif
            cmd_convert_thumb='convert ' + img_png + ' -resize 240x240 ' + img_thumb

            os.system(cmd_create_slices)
            os.system(cmd_convert_gif)
            os.system(cmd_convert_thumb)

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_gif)).insert(
            img_gif,
            content='ORIGINAL',
            overwrite=True,
            format='GIF')

            experiment.scan(scan_name).resource('SNAPSHOTS').file(os.path.basename(img_thumb)).insert(
            img_thumb,
            content='THUMBNAIL',
            overwrite=True,
            format='GIF')
                                                    

#xnat.select.project('demoProj_1').accessibility()
#xnat.select.project('demoProj_1').set_accessibility(accessibility='private')

#db.select.project('IXI_TEST').create()
print "ok"
    

OAS1_0027_MR1
disc1/OAS1_0027_MR1/RAW/OAS1_0027_MR1_mpr-1_anon.img
Now insert subject with key: OAS1_0027_MR1
OAS1_0027
OASIS_OAS1_0027_MR1
deleting scans
new hdr file is: disc1/OAS1_0027_MR1/RAW/OAS1_0027_MR1_mpr-1_anon.hdr
now flirting
OAS1_0332_MR1
disc9/OAS1_0332_MR1/RAW/OAS1_0332_MR1_mpr-1_anon.img
Now insert subject with key: OAS1_0332_MR1
OAS1_0332
OASIS_OAS1_0332_MR1
deleting scans
new hdr file is: disc9/OAS1_0332_MR1/RAW/OAS1_0332_MR1_mpr-1_anon.hdr
now flirting
OAS1_0162_MR1
disc5/OAS1_0162_MR1/RAW/OAS1_0162_MR1_mpr-1_anon.img
Now insert subject with key: OAS1_0162_MR1
OAS1_0162
OASIS_OAS1_0162_MR1
deleting scans
new hdr file is: disc5/OAS1_0162_MR1/RAW/OAS1_0162_MR1_mpr-1_anon.hdr
now flirting
OAS1_0270_MR1
disc7/OAS1_0270_MR1/RAW/OAS1_0270_MR1_mpr-1_anon.img
Now insert subject with key: OAS1_0270_MR1
OAS1_0270
OASIS_OAS1_0270_MR1
deleting scans
new hdr file is: disc7/OAS1_0270_MR1/RAW/OAS1_0270_MR1_mpr-1_anon.hdr
now flirting
OAS1_0022_MR1
disc1/OAS1_0022_MR1/RAW/OAS1_0022_